This notebook fit all the curve from any data (single or multiplex), <br>
HOWEVER IT IS NECESSARY TO SPECIFY THE TYPE OF DATA<br>
It will store all the dataframes in the data folder.<br>
Data generated will include:
- Normalised curves
- 5 sigm parameter (RAW CURVES)
- Fitted sigmoidal curves with all the datapoint (RAW CURVES)
- 5 sigm parameter (NORM CURVES)
- Fitted sigmoidal curves with all the datapoint (NORM CURVES)

# IMPORT LIBRARIES

In [1]:
from pathlib import Path

In [2]:
import python_libraries.sigmoid_fitting as fitfunc
import python_libraries.data_processing as procfunc
import python_libraries.data_loading as loadfunc
import python_libraries.utilities as utils

In [3]:
utils.colourblind()
NMETA = 6

# LOAD DATA
First we are loading all the data in the folder for that particular experiment.<br>
Obviously even here there is a date, inside the folder there might be multiple experiments, that's why after we create a dictionary which can downselect certain exp_ids.

In [4]:
data_path = f'{Path.cwd().parent}/data'
data_selected_path = f'{data_path}/20210716_7_plex'

In [5]:
singleplex_dict = {0: '20210706_01'}

multiplex_dict = {0: '20210707_01',
                  1: '20210707_03',
                  2: '20210707_02',
                  3: '20210708_01',
                  4: '20210708_02',
                  5: '20210708_03',
                  6: '20210715_01',
                  7: '20210715_02',
                  8: '20210721_02',
                  9: '20210721_03',
                 10: '20210722_01',
                 11: '20210723_01',
                 12: '20210723_02'}

DATA_TYPE = 'singleplex'
DATA_TYPE = 'multiplex'

Here we are passing the Key of the dictionary which corrisponds to a certain experiment ID.<br>
The data can be loaded from both Singleplex and Multiplex.<br>
The function "load_raw_data_by_expid( )", will take the path of the data and the list of seleceted exp_ids, to downselect them from the entire datafile.

In [7]:
# id_key = [0]
# id_key = [0, 1, 2, 3, 4, 5, 6, 7]
id_key = [11, 12]  # enter the exp_id from the dictionary above

exp_id_list = []  # storing all the exp_id selected

if DATA_TYPE == 'singleplex':
    exp_id_list = utils.list_from_key(singleplex_dict, id_key)

if DATA_TYPE == 'multiplex':
    exp_id_list = utils.list_from_key(multiplex_dict, id_key)
    
df_raw_rb = loadfunc.load_processed_data_by_expid(data_selected_path, exp_id_list, 'raw_rb')
df_raw_rb = utils.order_columns(df_raw_rb, NMETA)

print(f'Data shape: {df_raw_rb.shape}')
df_raw_rb.head(3)

Data shape: (119, 51)


,Channel,PrimerMix,Target,Assay,Conc,Exp_ID,1.0,2.0,3.0,4.0,...,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0
0,A01,PM7.2151,ADE,ADE_HEX_06,10000000,20210723_01,-0.456028,-0.456026,-0.455892,-0.453863,...,52330.887343,52531.874781,52683.762916,52799.227180,52887.526478,52955.451792,53008.008082,53048.903489,53080.900538,53106.068868
1,A02,PM7.2151,ADE,ADE_HEX_06,10000000,20210723_01,-0.479235,-0.479232,-0.479110,-0.477169,...,52922.305236,53079.420598,53197.493050,53286.792492,53354.759105,53406.811114,53446.916718,53477.999424,53502.226100,53521.212517
2,A03,PM7.2151,C22,C22_N_02,10000000,20210723_01,-0.026556,-0.026556,-0.026555,-0.026544,...,38440.782126,38449.311308,38455.108909,38459.089637,38461.849196,38463.779744,38465.142121,38466.111546,38466.806836,38467.309292


# NORMALIZED DATA based on FFI 
Divide all the values of the dataframe by the last cycle to let the DF goes from 0 to 1 as Fluorescence for each cycle

In [8]:
df_norm = procfunc.concat_norm_curve_on_FFI(df_raw_rb, NMETA)
df_norm.shape

(119, 51)

In [ ]:
# storing data in the folder (saving the new dataframe)
loadfunc.store_by_id(df_norm.reset_index(drop = True), data_selected_path, 'processed_data', 'norm_curve')

# SIGMOID FITTING (RAW)

## Pivot fitting for RAW CURVES

In [ ]:
bounds = (-100, 100)
maxfev = 1000000
core_number = 16
thresh_MSE = 0.0001
func = fitfunc.Five_Parameter_Sigmoid

In [ ]:
p0_initial = fitfunc.pivot_fitting(df_raw_rb, func,
                              NMETA, bounds, maxfev, core_number, thresh_MSE, seed=0)

print('Initial p0: ', p0_initial)

## Fit all data for RAW CURVES

In [ ]:
df_fitted_param, df_fitted_curve = fitfunc.sigmoid_fitting_parallel(df_raw_rb, 
                                                                    NMETA, func, p0_initial, 
                                                                    bounds, maxfev, core_number)

In [ ]:
# storing data in the folder (saving the new dataframe)
loadfunc.store_by_id(df_fitted_param.reset_index(drop = True), data_selected_path, 'processed_data', 'fitted_param')

In [ ]:
# storing data in the folder (saving the new dataframe)
loadfunc.store_by_id(df_fitted_curve.reset_index(drop = True), data_selected_path, 'processed_data', 'fitted_curve')

# SIGMOID FITTING (NORM)

## Pivot fitting for NORM CURVES

In [ ]:
p0_initial = fitfunc.pivot_fitting(df_norm, func,
                              NMETA, bounds, maxfev, core_number, thresh_MSE, seed=0)

print('Initial p0: ', p0_initial)

## Fit all the data for NORM CURVES

In [ ]:
df_fitted_param, df_fitted_curve = fitfunc.sigmoid_fitting_parallel(df_norm,
                                                                    NMETA, func, p0_initial, 
                                                                    bounds, maxfev, core_number)

In [ ]:
# storing data in the folder (saving the new dataframe)
loadfunc.store_by_id(df_fitted_param.reset_index(drop = True), data_selected_path, 'processed_data', 'norm_fitted_param')

In [ ]:
# storing data in the folder (saving the new dataframe)
loadfunc.store_by_id(df_fitted_curve.reset_index(drop = True), data_selected_path, 'processed_data', 'norm_fitted_curve')